In [1]:
# Email request from John T and Bruce 20191031

import pandas as pd
import numpy as np
import os
import datetime
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Experian_EasyLeasing'

In [2]:
df_experian=pd.read_table("/home/jian/BigLots/Experian/data_20191016/Experian_20191001114500.txt",sep="|",dtype=str)
print(df_experian.shape)

df_id_mapping=pd.read_table("/home/jian/BigLots/Experian/data_20191016/Experian 2019 customer hashed.txt",sep="|",dtype=str)
df_id_mapping=df_id_mapping.rename(columns={"Customer_Num":"CustomerNumber"})
print(df_id_mapping.shape)

df_experian=pd.merge(df_experian,df_id_mapping,on="CustomerNumber",how="left")
print(df_experian.shape)

(17626573, 19)
(17626573, 2)
(17626573, 20)


In [3]:
df_shopping_stores=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/output_2019-10-20/BL_allocated_store_by_id_shoppers_18_months_JL_20180415_20191012_JL_2019-10-20 19:46:00.089222.csv",
                              dtype=str,usecols=['customer_id_hashed','location_id'])

In [4]:
dfiddetail=pd.read_csv('/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/BL_all_master_id_email_zip_closet_store_ZipLabel_upto_20191012_JL_2019-10-21',
                       usecols=['customer_id_hashed','email_address_hash','closest_store'],dtype=str)

In [5]:
print(dfiddetail.shape,dfiddetail['customer_id_hashed'].nunique(),dfiddetail['email_address_hash'].nunique())

(30732606, 3) 30732606 30725689


In [6]:
print(df_shopping_stores.shape,df_shopping_stores['customer_id_hashed'].nunique())

(21589087, 2) 21589087


In [7]:
df_experian=df_experian[['CustomerNumber','IncomeId','IncomeDesc','AgeId','AgeDesc','hashed_customer_num']].rename(columns={"hashed_customer_num":"customer_id_hashed"})
print(df_experian.shape,df_experian['CustomerNumber'].nunique(),df_experian['customer_id_hashed'].nunique())

(17626573, 6) 17626573 17626573


In [8]:
df_experian=pd.merge(df_experian,dfiddetail,on="customer_id_hashed",how="left")
df_experian=pd.merge(df_experian,df_shopping_stores,on="customer_id_hashed",how="left")
print(df_experian.shape)
df_experian.head(2)

(17626573, 9)


,CustomerNumber,IncomeId,IncomeDesc,AgeId,AgeDesc,customer_id_hashed,email_address_hash,closest_store,location_id
0,9,2,between $50k and $100k,2,between 35 and 50,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebab...,dd53075d06b1c045bb7b5756adf7e39d1ef26b24022b17...,1468,1755
1,28,3,>= $100k,3,>= 50,59e19706d51d39f66711c2653cd7eb1291c94d9b55eb14...,bbe01b57dc8fb9c8c9f6722f15836c73ba8eb481279af0...,4660,4660


In [9]:
df_experian['final_associate_store']=np.where(pd.notnull(df_experian['location_id']),df_experian['location_id'],df_experian['closest_store'])


In [10]:
# SOTF & Legacy

ExcelFile=pd.ExcelFile("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Copy of BL 2019Q4 Budget Allocation Model 20191018_JC_JL_20191020.xlsx")
df_SOTF=ExcelFile.parse("4_SOTF",usecols=["location_id","copy_SOTF_label"],dtype=str)
df_SOTF=df_SOTF.rename(columns={"copy_SOTF_label":"SOTF","location_id":"final_associate_store"})
df_SOTF['SOTF']=df_SOTF['SOTF'].replace("nan",np.nan)

print(df_SOTF.shape)
df_SOTF.head(3)

(1433, 2)


,final_associate_store,SOTF
0,1918,SOTF
1,4668,SOTF
2,4674,SOTF


In [11]:
df_experian=pd.merge(df_experian,df_SOTF,on="final_associate_store",how="left")
df_experian.shape

(17626573, 11)

In [12]:
df_experian['SOTF']=df_experian['SOTF'].fillna("Legacy")

In [13]:
df_experian.head(2)

,CustomerNumber,IncomeId,IncomeDesc,AgeId,AgeDesc,customer_id_hashed,email_address_hash,closest_store,location_id,final_associate_store,SOTF
0,9,2,between $50k and $100k,2,between 35 and 50,19581e27de7ced00ff1ce50b2047e7a567c76b1cbaebab...,dd53075d06b1c045bb7b5756adf7e39d1ef26b24022b17...,1468,1755,1755,SOTF
1,28,3,>= $100k,3,>= 50,59e19706d51d39f66711c2653cd7eb1291c94d9b55eb14...,bbe01b57dc8fb9c8c9f6722f15836c73ba8eb481279af0...,4660,4660,4660,Legacy


In [14]:
# Uploaded regular Q4 audience

list_uploaded_audience=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/uploaded_no_T_zip/*.csv")
print(len(list_uploaded_audience))
list_uploaded_audience_T=[x for x in list_uploaded_audience if os.path.basename(x)[0]=="T"]
list_uploaded_audience_C=[x for x in list_uploaded_audience if os.path.basename(x)[0]=="C"]
print(len(list_uploaded_audience_T)+len(list_uploaded_audience_C))

list_df_test_Q4_audience=[]
list_df_control_Q4_audience=[]


for file in list_uploaded_audience_T:
    df=pd.read_csv(file,dtype=str)
    list_df_test_Q4_audience.append(df)
    
for file in list_uploaded_audience_C:
    df=pd.read_csv(file,dtype=str)
    list_df_control_Q4_audience.append(df)

df_test_Q4_audience=pd.concat(list_df_test_Q4_audience)
df_control_Q4_audience=pd.concat(list_df_control_Q4_audience)

print(df_test_Q4_audience.shape,df_control_Q4_audience.shape)

178
178
(21435770, 3) (541818, 3)


In [15]:
del list_df_test_Q4_audience
del list_df_control_Q4_audience
del df

gc.collect()

77

In [16]:
df_test_Q4_audience.head(2)

,customer_id_hashed,email_address_hash,segment
0,de4d254f85f84828db782a91857b389542ca253714330b...,55759d0e688507ce244ad5c86f8a71d4f4f54e0873452d...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...
1,ea4e3e1699773386da4e152d8b9b93d10e21383ed231e0...,1214d3d086f42b075b04ab09839c170969c575653da33e...,T_Active (0-12)_Score(H)_Density(H)_Legacy_Hol...


In [17]:
Q4_test_ids=df_test_Q4_audience['customer_id_hashed'].unique().tolist()
Q4_test_email=df_test_Q4_audience['email_address_hash'].unique().tolist()

Q4_control_ids=df_control_Q4_audience['customer_id_hashed'].unique().tolist()
Q4_control_email=df_control_Q4_audience['email_address_hash'].unique().tolist() # customer_id_hashed

In [18]:
df_experian['Q4_group_uploaded']=np.where((df_experian['customer_id_hashed'].isin(Q4_control_ids) | df_experian['email_address_hash'].isin(Q4_control_email) ),"Q4_Control",
                                         np.where((df_experian['customer_id_hashed'].isin(Q4_test_ids))|(df_experian['email_address_hash'].isin(Q4_test_email) ),"Q4_Test","Others")
                                         )

In [19]:
df_summary_original=df_experian.groupby(['SOTF','IncomeId','IncomeDesc','AgeId','AgeDesc','Q4_group_uploaded'])['customer_id_hashed','email_address_hash'].nunique().reset_index()
df_summary_original=df_summary_original.rename(columns={"customer_id_hashed":"id_count","email_address_hash":"email_count"})


In [20]:
df_summary_original.shape

(54, 8)

In [21]:
'''
SOTF - HHI Income: <$50k
SOTF - HHI: $50k - $100k & <50 yr old
LEGACY - HHI Income: <$50k
LEGACY - HHI: $50k - $100k & <50 yr old
'''

'\nSOTF - HHI Income: <$50k\nSOTF - HHI: $50k - $100k & <50 yr old\nLEGACY - HHI Income: <$50k\nLEGACY - HHI: $50k - $100k & <50 yr old\n'

In [22]:
df_experian['AgeDesc'].unique()

array(['between 35 and 50', '>= 50', '< 35', nan], dtype=object)

In [23]:
print(datetime.datetime.now())

2019-10-31 15:29:01.265608


In [24]:
seg_1_SOTF_HHI_LowerThan50K=df_experian[(df_experian['SOTF']=="SOTF") & (df_experian['IncomeDesc']=="< $50k")]
seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50=df_experian[(df_experian['SOTF']=="SOTF") & (df_experian['IncomeDesc']=="between $50k and $100k") & (df_experian['AgeDesc'].isin(['between 35 and 50','< 35']))]

seg_3_Legacy_HHI_LowerThan50K=df_experian[(df_experian['SOTF']=="Legacy") & (df_experian['IncomeDesc']=="< $50k")]
seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50=df_experian[(df_experian['SOTF']=="Legacy") & (df_experian['IncomeDesc']=="between $50k and $100k") & (df_experian['AgeDesc'].isin(['between 35 and 50','< 35']))]

print("seg_1_SOTF_HHI_LowerThan50K: ",seg_1_SOTF_HHI_LowerThan50K.shape,seg_1_SOTF_HHI_LowerThan50K['customer_id_hashed'].nunique(),seg_1_SOTF_HHI_LowerThan50K['email_address_hash'].nunique())
print("seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50: ",seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50.shape,seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50['customer_id_hashed'].nunique(),seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50['email_address_hash'].nunique())
print("seg_3_Legacy_HHI_LowerThan50K: ",seg_3_Legacy_HHI_LowerThan50K.shape,seg_3_Legacy_HHI_LowerThan50K['customer_id_hashed'].nunique(),seg_3_Legacy_HHI_LowerThan50K['email_address_hash'].nunique())
print("seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50: ",seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50.shape,seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50['customer_id_hashed'].nunique(),seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50['email_address_hash'].nunique())


seg_1_SOTF_HHI_LowerThan50K:  (1401254, 12) 1401254 1258115
seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50:  (1494770, 12) 1494770 1360902
seg_3_Legacy_HHI_LowerThan50K:  (3116742, 12) 3116742 2800953
seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50:  (3296127, 12) 3296127 3015026


In [25]:
gc.collect()

0

In [26]:
writer_folder="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Experian_EasyLeasing/"

try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)
    

In [27]:
# Remove all nan emails

seg_1_SOTF_HHI_LowerThan50K=seg_1_SOTF_HHI_LowerThan50K[pd.notnull(seg_1_SOTF_HHI_LowerThan50K['email_address_hash'])]
seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50=seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50[pd.notnull(seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50['email_address_hash'])]
seg_3_Legacy_HHI_LowerThan50K=seg_3_Legacy_HHI_LowerThan50K[pd.notnull(seg_3_Legacy_HHI_LowerThan50K['email_address_hash'])]
seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50=seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50[pd.notnull(seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50['email_address_hash'])]

seg_1_summary=seg_1_SOTF_HHI_LowerThan50K.groupby(['SOTF','IncomeDesc','AgeDesc','Q4_group_uploaded'])['email_address_hash'].nunique().to_frame().reset_index()
seg_2_summary=seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50.groupby(['SOTF','IncomeDesc','AgeDesc','Q4_group_uploaded'])['email_address_hash'].nunique().to_frame().reset_index()
seg_3_summary=seg_3_Legacy_HHI_LowerThan50K.groupby(['SOTF','IncomeDesc','AgeDesc','Q4_group_uploaded'])['email_address_hash'].nunique().to_frame().reset_index()
seg_4_summary=seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50.groupby(['SOTF','IncomeDesc','AgeDesc','Q4_group_uploaded'])['email_address_hash'].nunique().to_frame().reset_index()


In [28]:
writer=pd.ExcelWriter(writer_folder+"BL_Experian_EasyLeasing_Summary_before_removing_Q4_control_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary_original.to_excel(writer,"summary_original",index=False)
seg_1_summary.to_excel(writer,"SOTF_HHI_LowerThan50K",index=False)
seg_2_summary.to_excel(writer,"SOTF_HHI_50Kto100K_AgeLower50",index=False)
seg_3_summary.to_excel(writer,"Legacy_HHI_LowerThan50K",index=False)
seg_4_summary.to_excel(writer,"Legacy_HHI_50Kto100K_AgeLower50",index=False)
writer.save()

In [29]:
# Remove all Q4 control ids

df_output_1=seg_1_SOTF_HHI_LowerThan50K[seg_1_SOTF_HHI_LowerThan50K['Q4_group_uploaded']!="Q4_Control"]
df_output_2=seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50[seg_2_SOTF_HHI_50Kto100K_AgeLowerThan50['Q4_group_uploaded']!="Q4_Control"]
df_output_3=seg_3_Legacy_HHI_LowerThan50K[seg_3_Legacy_HHI_LowerThan50K['Q4_group_uploaded']!="Q4_Control"]
df_output_4=seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50[seg_4_Legacy_HHI_50Kto100K_AgeLowerThan50['Q4_group_uploaded']!="Q4_Control"]




In [30]:
df_output_1=df_output_1[['customer_id_hashed','email_address_hash']]
df_output_1['segment']="Experian_EasyLeasing_SOTF_HHI50KLower"

df_output_2=df_output_2[['customer_id_hashed','email_address_hash']]
df_output_2['segment']="Experian_EasyLeasing_SOTF_HHI50Kto100K_Age50Lower"

df_output_3=df_output_3[['customer_id_hashed','email_address_hash']]
df_output_3['segment']="Experian_EasyLeasing_Legacy_HHI50KLower"

df_output_4=df_output_4[['customer_id_hashed','email_address_hash']]
df_output_4['segment']="Experian_EasyLeasing_Legacy_HHI50Kto100K_Age50Lower"


In [31]:
writer_folder_uploaded="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q4/Experian_EasyLeasing/uploaded/"

try:
    os.stat(writer_folder_uploaded)
except:
    os.mkdir(writer_folder_uploaded)
    

In [32]:
df_output_1.to_csv(writer_folder_uploaded+"Experian_EasyLeasing_SOTF_HHI50KLower.csv",index=False)
df_output_2.to_csv(writer_folder_uploaded+"Experian_EasyLeasing_SOTF_HHI50Kto100K_Age50Lower.csv",index=False)
df_output_3.to_csv(writer_folder_uploaded+"Experian_EasyLeasing_Legacy_HHI50KLower.csv",index=False)
df_output_4.to_csv(writer_folder_uploaded+"Experian_EasyLeasing_Legacy_HHI50Kto100K_Age50Lower.csv",index=False)

In [33]:
print(df_output_1['segment'].unique(),df_output_1.shape,df_output_1['customer_id_hashed'].nunique(),df_output_1['email_address_hash'].nunique())
print(df_output_2['segment'].unique(),df_output_2.shape,df_output_2['customer_id_hashed'].nunique(),df_output_2['email_address_hash'].nunique())
print(df_output_3['segment'].unique(),df_output_3.shape,df_output_3['customer_id_hashed'].nunique(),df_output_3['email_address_hash'].nunique())
print(df_output_4['segment'].unique(),df_output_4.shape,df_output_4['customer_id_hashed'].nunique(),df_output_4['email_address_hash'].nunique())


['Experian_EasyLeasing_SOTF_HHI50KLower'] (1229208, 3) 1229208 1229146
['Experian_EasyLeasing_SOTF_HHI50Kto100K_Age50Lower'] (1328707, 3) 1328707 1328676
['Experian_EasyLeasing_Legacy_HHI50KLower'] (2738480, 3) 2738480 2738360
['Experian_EasyLeasing_Legacy_HHI50Kto100K_Age50Lower'] (2945147, 3) 2945147 2945085
